## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Pisco,PE,-13.7000,-76.2167,21.03,78,75,7.20,broken clouds
1,1,Hilo,US,19.7297,-155.0900,23.85,91,90,5.66,heavy intensity rain
2,2,Nikolskoye,RU,59.7035,30.7861,-17.56,98,8,2.14,clear sky
3,3,Vaini,TO,-21.2000,-175.2000,27.09,89,75,6.17,broken clouds
4,4,Atuona,PF,-9.8000,-139.0333,25.83,75,5,8.18,clear sky


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input('What is the minimum temperature you prefer?'))
max_temp = int(input('What is the maximum temperature you prefer?'))

What is the minimum temperature you prefer?25
What is the maximum temperature you prefer?40


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
pref_cities = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                 (city_data_df['Max Temp'] >= min_temp)]
pref_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Vaini,TO,-21.2000,-175.2000,27.09,89,75,6.17,broken clouds
4,4,Atuona,PF,-9.8000,-139.0333,25.83,75,5,8.18,clear sky
5,5,Georgetown,MY,5.4112,100.3354,25.50,77,20,4.63,few clouds
7,7,Porto Velho,BR,-8.7619,-63.9039,28.03,78,20,1.03,few clouds
8,8,Lucea,JM,18.4510,-78.1736,27.92,76,90,3.10,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
673,673,Newport Beach,US,33.6189,-117.9289,25.95,70,1,0.89,haze
678,678,Neiafu,TO,-18.6500,-173.9833,26.38,94,90,8.74,light rain
685,685,Maxixe,MZ,-23.8597,35.3472,25.31,81,0,5.05,clear sky
686,686,Yenagoa,NG,4.9247,6.2642,25.50,89,100,2.23,overcast clouds


In [17]:
# 4a. Determine if there are any empty rows.
pref_cities.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities = pref_cities.dropna()
pref_cities.count()

City_ID        167
City           167
Country        167
Lat            167
Lng            167
Max Temp       167
Humidity       167
Cloudiness     167
Wind Speed     167
Description    167
dtype: int64

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Vaini,TO,27.09,broken clouds,-21.2000,-175.2000,
4,Atuona,PF,25.83,clear sky,-9.8000,-139.0333,
5,Georgetown,MY,25.50,few clouds,5.4112,100.3354,
7,Porto Velho,BR,28.03,few clouds,-8.7619,-63.9039,
8,Lucea,JM,27.92,overcast clouds,18.4510,-78.1736,
11,Avarua,CK,28.03,moderate rain,-21.2078,-159.7750,
13,Waw,SS,28.14,broken clouds,7.7029,27.9953,
19,Puerto Leguizamo,CO,25.28,heavy intensity rain,-0.1934,-74.7819,
30,Hithadhoo,MV,27.35,overcast clouds,-0.6000,73.0833,
33,Forecariah,GN,25.64,scattered clouds,9.4306,-13.0881,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('No hotel found... skipping!')
        

No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!


In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df[hotel_df['Hotel Name'] == '']
hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
hotel_df.count()

City           158
Country        158
Max Temp       158
Description    158
Lat            158
Lng            158
Hotel Name     158
dtype: int64

In [22]:
# 8a. Create the output File (CSV)
output_data_file = 'Weather_Database/Hotels.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))